In [3]:
import psycopg2 as pg
import pandas.io.sql as pd_sql
import pandas as pd
import io
import requests
from sqlalchemy import create_engine

### Connect to AWS

In [10]:
# https://github.com/thisismetis/sf20_ds17/blob/master/curriculum/project-03/_
### aws-setup/02_HW_setup_aws_and_psycopg.md
# to update AWS server to hear my IP
# sudo su - postgres
# nano /etc/postgresql/10/main/pg_hba.conf
# Search for "IPv4 local connections"
# type exti to stop being user postgres
# restart psql: sudo service postgresql restart

connection_args = {
    'host': '13.52.182.91', #AWS's IP here
    'user': 'ubuntu',
    'dbname': 'hurricane',
    'port': 5432
}

connection = pg.connect(**connection_args)

OperationalError: could not connect to server: Connection timed out (0x0000274C/10060)
	Is the server running on host "13.52.182.91" and accepting
	TCP/IP connections on port 5432?


### Get Data from NOAA
National Oceanic and Atmospheric Administration

In [5]:
url="https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2018-120319.txt"
headers = ['Year_Month_Day', 'Hour_Minute', 'Status'
           , 'Classification', 'Latitude', 'Longitude', 'Max_sustained_wind_knots', 'Minimum_pressure_millibars'
           , '34ktNEMaxRadii_nautmiles', '34ktSEMaxRadii_nautmiles', '34ktSWMaxRadii_nautmiles', '34ktNWMaxRadii_nautmiles'
           , '50ktNEMaxRadii_nautmiles', '50ktSEMaxRadii_nautmiles', '50ktSWMaxRadii_nautmiles', '50ktNWMaxRadii_nautmiles'
           , '64ktNEMaxRadii_nautmiles', '64ktSEMaxRadii_nautmiles', '64ktSWMaxRadii_nautmiles', '64ktNWMaxRadii_nautmiles'
          ]
s=requests.get(url).content
data=pd.read_csv(io.StringIO(s.decode('utf-8')), names=headers)

### Upload NOAA to AWS Database
#### This takes a few minutes to run

In [6]:
query= """
    CREATE TABLE hurricane (
        Year_Month_Day varchar(200),
        Hour_Minute varchar(200),
        Status varchar(200),
        Classification varchar(200),
        Latitude varchar(200),
        Longitude varchar(200),
        Max_sustained_wind_knots float,
        Minimum_pressure_millibars float,
        thirtyfour_ktNEMaxRadii_nautmiles float,
        thirtyfour_ktSEMaxRadii_nautmiles float,
        thirtyfour_ktSWMaxRadii_nautmiles float,
        thirtyfour_ktNWMaxRadii_nautmiles float,
        fifty_ktNEMaxRadii_nautmiles float,
        fifty_ktSEMaxRadii_nautmiles float,
        fifty_ktSWMaxRadii_nautmiles float,
        fifty_ktNWMaxRadii_nautmiles float,
        sixtyfour_ktNEMaxRadii_nautmiles float,
        sixtyfour_ktSEMaxRadii_nautmiles float,
        sixtyfour_ktSWMaxRadii_nautmiles float,
        sixtyfour_ktNWMaxRadii_nautmiles float
    );
"""

engine = create_engine('postgresql://ubuntu@13.52.182.91:5432/hurricane')
data.to_sql('hurricane', engine, if_exists='replace', index=False)